# Data Preparation

## PDF-Text extrahieren & vorbereiten

Der Nachhaltigkeitsbericht wird eingelesen, gereinigt und in Abschnitte (Chunks) unterteilt.

In [1]:
import fitz
import re
import unicodedata

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# def clean_text(text):
#     text = text.replace("\n", " ")
#     text = re.sub(r'\s+', ' ', text)  # Mehrfache Leerzeichen entfernen
#     return text.strip()

def clean_text(text):
    # Unicode-Normalisierung (z. B. Ligaturen auflösen)
    text = unicodedata.normalize("NFKC", text)

    # Entferne weiche Trennzeichen (wie "\xad")
    text = text.replace('\xad', '')

    # Entferne PDF-Silbentrennungen: z. B. „Transfor-\n mation“ → „Transformation“
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)

    # Ersetze Zeilenumbrüche mit Leerzeichen
    text = text.replace("\n", " ")

    # Entferne Steuerzeichen und Sonderzeichen
    text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')

    # Entferne doppelte Leerzeichen
    text = re.sub(r'\s+', ' ', text)

    # Korrigiere Leerzeichen vor Satzzeichen
    text = re.sub(r'\s+([.,;:!?])', r'\1', text)

    return text.strip()

raw_text = extract_text_from_pdf("reports/2023_Volkswagen_Group_Nachhaltigkeitsbericht.pdf")
text = clean_text(raw_text)

## Text in Chunks aufteilen

In [2]:
# def chunk_text(text, max_length=500):
#     sentences = re.split(r'(?<=[.!?])\s+', text)
#     chunks, current_chunk = [], ""
    
#     for sentence in sentences:
#         if len(current_chunk) + len(sentence) <= max_length:
#             current_chunk += " " + sentence
#         else:
#             chunks.append(current_chunk.strip())
#             current_chunk = sentence
#     if current_chunk:
#         chunks.append(current_chunk.strip())
    
#     return chunks

# chunks = chunk_text(text)
# print(f"Anzahl Chunks: {len(chunks)}")

In [3]:
# chunks[1]

In [4]:
def split_sentences(text):
    # Schützt gängige Abkürzungen vor falscher Trennung
    protected = {
        r'\bz\. *B\.': 'z.B.',
        r'\bu\. *a\.': 'u.a.',
        r'\bu\. *Ä\.': 'u.Ä.',
        r'\bd\. *h\.': 'd.h.',
        r'\bu\. *s\. *w\.': 'usw.',
    }
    for pattern, replacement in protected.items():
        text = re.sub(pattern, replacement, text)

    # Trenne nach Punkt, Ausrufezeichen oder Fragezeichen + Leerzeichen + Großbuchstabe
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-ZÄÖÜ])', text)

    return [s.strip() for s in sentences if s.strip()]

def group_sentences(sentences, max_length=500):
    chunks, current = [], ""

    for sentence in sentences:
        if len(current) + len(sentence) + 1 <= max_length:  # +1 für Leerzeichen
            if current:
                current += " " + sentence
            else:
                current = sentence
        else:
            chunks.append(current.strip())
            current = sentence
    if current:
        chunks.append(current.strip())

    return chunks

def split_and_chunk_text(text, max_length=500):
    sentences = split_sentences(text)
    chunks = group_sentences(sentences, max_length)
    return chunks

# Anwendung (ersetzt chunk_text)
chunks = split_and_chunk_text(text)

print(f"Anzahl Chunks: {len(chunks)}")
print(f"Beispiel Chunk:\n{chunks[1]}")


Anzahl Chunks: 1164
Beispiel Chunk:
2023 N A C H H A LT I G K E I T S B E R I C H T Inhalt Navigationshilfe Kapitelverweise innerhalb des Nachhaltigkeitsberichts Verweise auf Websites 6 Vorwort 9 Über den Bericht 12 Nachhaltigkeit in der Unternehmens-DNA 13 Corporate Governance 17 Konzernstrategie NEW AUTO 22 ESG-Performance-Management und Wesentlichkeitsanalyse 25 Nachhaltigkeitsmanagement 27 Nachhaltigkeitswirkungen neuer Mobilitätsangebote 30 Stakeholder-Management 33 Umwelt-Compliance-Management 42 Risikomanagement 45 Corporate Citizenship Fokusthemen 1 Dekarbonisierung 48 Dekarbonisierung 69 EU-Taxonomie 2 Kreislaufwirtschaft 83 Kreislaufwirtschaft 3 Menschen in der Transformation 90 Menschen in der Transformation 4 Vielfalt 104 Vielfalt 5 Integrität und Compliance 109 Integrität und Compliance 6 Lieferkette und Menschenrechte 117 Lieferkette und Menschenrechte 131 Anhang 132 Vermerk des unabhängigen Wirtschaftsprüfers 135 Impressum 2 Nachhaltigkeits kommunikation des Volkswagen K

## Textverarbeitung

Tokenisierung, Stoppwörter & Lemmatisierung

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#nltk.download('stopwords')
nltk.download('wordnet')

#stop_words = set(stopwords.words('german'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zäöüß ]', ' ', text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if len(token) > 2]
    #tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2]
    return " ".join(tokens)

preprocessed_chunks = [preprocess_text(chunk) for chunk in chunks]
print(f"Beispiel vorverarbeiteter Chunk:\n{preprocessed_chunks[1][:100]}...\n")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\schmu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Beispiel vorverarbeiteter Chunk:
inhalt navigationshilfe kapitelverweise innerhalb de nachhaltigkeitsberichts verweise auf website vo...



## Filterung von Nachhaltigkeitsversprechen

In diesem Schritt werden die Textabschnitte (Chunks) auf das Vorkommen vordefinierter Schlüsselbegriffe zu Nachhaltigkeit (z. B. klimaneutral, emissionen, paris) überprüft. Nur die Textpassagen, die mindestens einen dieser Begriffe enthalten, werden als potenzielle Nachhaltigkeitsversprechen markiert und für die weitere Analyse berücksichtigt.

In [6]:
relevant_keywords = [ 
    "verantwortung", "nachhaltigkeit", "dekarbonisierung", "strategie",
    "2030", "klimaschutz", "emission",  "deklarieren", "klimaziel", 
    "co2", "paris", "klimaneutralität", "umweltziel"
]

# Keywords normalisiert (klein & lemmatisiert)
lemmatized_keywords = [lemmatizer.lemmatize(w.lower()) for w in relevant_keywords]

def contains_keywords(text):
    return any(keyword in text for keyword in lemmatized_keywords)

print("Keywords:", lemmatized_keywords)
print("Original Chunk Beispiel:\n", chunks[1][:300])
print("Vorverarbeiteter Chunk Beispiel:\n", preprocessed_chunks[1][:300])

# 2. Check Filterfunktion:
print("Enthält Keywords?", contains_keywords(preprocessed_chunks[1]))

# 3. Filterung mit originalen und vorverarbeiteten Chunks verknüpfen
filtered_chunks = [
    original for original, preprocessed in zip(chunks, preprocessed_chunks)
    if contains_keywords(preprocessed)
]

print(f"Gefilterte Chunks: {len(filtered_chunks)}")
print("Beispiel gefilterter Chunk:", filtered_chunks[1][:300])

Keywords: ['verantwortung', 'nachhaltigkeit', 'dekarbonisierung', 'strategie', '2030', 'klimaschutz', 'emission', 'deklarieren', 'klimaziel', 'co2', 'paris', 'klimaneutralität', 'umweltziel']
Original Chunk Beispiel:
 2023 N A C H H A LT I G K E I T S B E R I C H T Inhalt Navigationshilfe Kapitelverweise innerhalb des Nachhaltigkeitsberichts Verweise auf Websites 6 Vorwort 9 Über den Bericht 12 Nachhaltigkeit in der Unternehmens-DNA 13 Corporate Governance 17 Konzernstrategie NEW AUTO 22 ESG-Performance-Managemen
Vorverarbeiteter Chunk Beispiel:
 inhalt navigationshilfe kapitelverweise innerhalb de nachhaltigkeitsberichts verweise auf website vorwort über den bericht nachhaltigkeit der unternehmens dna corporate governance konzernstrategie new auto esg performance management und wesentlichkeitsanalyse nachhaltigkeitsmanagement nachhaltigkeit
Enthält Keywords? True
Gefilterte Chunks: 465
Beispiel gefilterter Chunk: Auf unserem Weg vom Automobilhersteller zum Mobilitätskonzern ordnen wir

## KeyBERT zur Extraktion der relevantesten Aussagen

In diesem Schritt wird mithilfe von KeyBERT und einem vortrainierten SentenceTransformer-Modell (MiniLM) aus jedem Textabschnitt (chunk) eine Liste der wichtigsten Schlüsselbegriffe (Keywords) extrahiert. Diese Keywords dienen als Grundlage für die semantische Analyse und spätere Filterung von relevanten Nachhaltigkeitsaussagen.

In [7]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
kw_model = KeyBERT(model=model)

#german_stopwords = stopwords.words('german')

for i, chunk in enumerate(filtered_chunks[:5]):
    #keywords = kw_model.extract_keywords(chunk, keyphrase_ngram_range=(1, 3), stop_words=german_stopwords, top_n=5)
    keywords = kw_model.extract_keywords(chunk, keyphrase_ngram_range=(1, 3), stop_words=None, top_n=5)
    print(f"Chunk {i+1} Keywords: {keywords}\n")

c:\Dokumente\Uni Master\SoSe25\Unternehmenssoftware\Projekt\Greenwashing_Detector\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Chunk 1 Keywords: [('nachhaltigkeitsmanagement 27 nachhaltigkeitswirkungen', 0.6765), ('nachhaltigkeitsmanagement 27', 0.6708), ('nachhaltigkeitsmanagement', 0.6663), ('25 nachhaltigkeitsmanagement', 0.6639), ('wesentlichkeitsanalyse 25 nachhaltigkeitsmanagement', 0.6635)]

Chunk 2 Keywords: [('automobilhersteller zum mobilitätskonzern', 0.6103), ('vom automobilhersteller zum', 0.5862), ('uns zukunftsgerichtet auf', 0.5501), ('weg vom automobilhersteller', 0.5497), ('mobilitätskonzern ordnen wir', 0.5488)]

Chunk 3 Keywords: [('der vorliegende nachhaltigkeitsbericht', 0.6895), ('bereich nachhaltigkeit und', 0.6495), ('den bereich nachhaltigkeit', 0.614), ('vorliegende nachhaltigkeitsbericht', 0.6078), ('nachhaltigkeit und esg', 0.5929)]

Chunk 4 Keywords: [('der unternehmens dna', 0.6111), ('unternehmens dna eine', 0.6088), ('das unternehmen und', 0.607), ('unternehmen und ein', 0.5997), ('das unternehmen', 0.5867)]

Chunk 5 Keywords: [('co2 fußabdruck des', 0.515), ('energy volkswagen

In [8]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

preprocessed_filtered_chunks = [preprocess_text(chunk) for chunk in filtered_chunks]

embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
topic_model = BERTopic(language="german", embedding_model=embedding_model)

topics, probs = topic_model.fit_transform(preprocessed_filtered_chunks)

topic_info = topic_model.get_topic_info()
print(topic_info)

print("\nBeispiel Topic 1:")
print(topic_model.get_topic(1))

   Topic  Count                             Name  \
0      0    340                0_der_und_die_für   
1      1    125  1_und_der_compliance_integrität   

                                      Representation  \
0  [der, und, die, für, de, den, von, auf, mit, v...   
1  [und, der, compliance, integrität, vielfalt, m...   

                                 Representative_Docs  
0  [nicht bestandteil de zusammengefassten nichtf...  
1  [inhalt unternehmens dna dekarbonisierung krei...  

Beispiel Topic 1:
[('und', 0.15065432023213002), ('der', 0.10454371427599796), ('compliance', 0.08656847103301095), ('integrität', 0.0800915530674448), ('vielfalt', 0.07948936093911815), ('menschen', 0.07936847333949103), ('transformation', 0.07866007464375946), ('menschenrechte', 0.07864804305796894), ('kreislaufwirtschaft', 0.0782953797101365), ('dekarbonisierung', 0.07706526680016769)]


## Artikel Vorverarbeitung

In [9]:
from newspaper import Article

url = "https://nachhaltigkeit-wirtschaft.de/nachhaltigkeitsbericht-volkswagen-2022-wie-wir-den-wandel-zu-nachhaltiger-mobilitaet-vorantreiben/"
article = Article(url, language='de')
article.download()
article.parse()
article_text = article.text

print(article_text)

Nachhaltigkeitsbericht Volkswagen 2022: Strategische Bedeutung für den Konzern

Nachhaltigkeitsbericht Volkswagen 2022: Strategische Bedeutung für den Konzern

Werbung

Der Nachhaltigkeitsbericht Volkswagen 2022 markiert einen Wendepunkt in der Unternehmensstrategie des Konzerns. Nachhaltigkeit ist längst kein Randthema mehr, sondern zieht sich wie ein roter Faden durch sämtliche Geschäftsbereiche. Im Bericht wird deutlich, dass Volkswagen den Wandel zur nachhaltigen Mobilität nicht nur als Reaktion auf gesellschaftliche Erwartungen versteht, sondern als aktiven Gestaltungsauftrag. Die Integration von Nachhaltigkeitszielen in alle Steuerungs- und Entscheidungsprozesse sorgt dafür, dass ökologische und soziale Verantwortung auf Vorstandsebene beginnt und bis in die operative Umsetzung reicht.

Erweitern Sie Ihr Wissen! Lesen Sie relevante Fachliteratur zum Thema Nachhaltigkeit und lernen Sie verschiedene Ansätze und Ideen zum Umgang mit Nachhaltigkeitsthemen kennen. Jetzt mehr erfahren 

In [10]:
cleaned_article_text = clean_text(article_text)
#article_chunks = chunk_text(cleaned_article_text)
#print(f"Anzahl Artikel-Chunks: {len(article_chunks)}")
article_sentences = split_sentences(cleaned_article_text)
print(f"Anzahl Artikel-Sätze: {len(article_sentences)}")

Anzahl Artikel-Sätze: 69


In [11]:
#preprocessed_article_chunks = [preprocess_text(chunk) for chunk in article_chunks]
preprocessed_article_chunks = [preprocess_text(chunk) for chunk in article_sentences]

print(f"Beispiel vorverarbeiteter Chunk:\n{preprocessed_article_chunks[0][:200]}")

Beispiel vorverarbeiteter Chunk:
nachhaltigkeitsbericht volkswagen strategische bedeutung für den konzern nachhaltigkeitsbericht volkswagen strategische bedeutung für den konzern werbung der nachhaltigkeitsbericht volkswagen markiert


In [12]:
#für kleine Datenmengen
from umap import UMAP
import hdbscan

embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

umap_model = UMAP(n_neighbors=3, n_components=5, metric='cosine')
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=2, metric='euclidean', prediction_data=True)

topic_model = BERTopic(
    language="german",
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model
)

topics, probs = topic_model.fit_transform(preprocessed_article_chunks)

topic_info = topic_model.get_topic_info()
print(topic_info)

print("\nBeispiel Topic 1:")
print(topic_model.get_topic(1))

   Topic  Count                                        Name  \
0      0     33                    0_volkswagen_und_der_die   
1      1     33                        1_und_die_der_werden   
2      2      3  2_investoren_und_herausforderungen_bereich   

                                      Representation  \
0  [volkswagen, und, der, die, nachhaltigkeitsber...   
1  [und, die, der, werden, für, von, auf, ein, al...   
2  [investoren, und, herausforderungen, bereich, ...   

                                 Representative_Docs  
0  [nachhaltigkeitsbericht volkswagen strategisch...  
1  [externe esg rating und berichte von organisat...  
2  [die integration der anforderungen au hgb und ...  

Beispiel Topic 1:
[('und', 0.12466400859322543), ('die', 0.09219768084425568), ('der', 0.0778407317954388), ('werden', 0.07132081587333147), ('für', 0.05037458879003155), ('von', 0.04582677107195985), ('auf', 0.044979185139965606), ('ein', 0.039966237326984996), ('al', 0.0392800894902513), ('sondern'

# Modeling

## Vergleich von Artikel und Bericht

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

all_report_embeddings = embedding_model.encode(preprocessed_filtered_chunks, convert_to_tensor=True)
all_article_embeddings = embedding_model.encode(preprocessed_article_chunks, convert_to_tensor=True)

similarity_matrix = cosine_similarity(all_article_embeddings, all_report_embeddings)

top_k = 3

for i, article_chunk in enumerate(preprocessed_article_chunks):
    similarities = similarity_matrix[i]
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    #print(f"\nArtikel-Chunk {i+1}:\n{article_chunks[i][:300]}...\n")
    print(f"\nArtikel-Chunk {i+1}:\n{article_sentences[i][:300]}...\n")

    for rank, idx in enumerate(top_indices):
        print(f"Ähnlichster Bericht-Chunk {rank+1} (Score: {similarities[idx]:.2f}):")
        print(filtered_chunks[idx][:300])
        print("-" * 80)


Artikel-Chunk 1:
Nachhaltigkeitsbericht Volkswagen 2022: Strategische Bedeutung für den Konzern Nachhaltigkeitsbericht Volkswagen 2022: Strategische Bedeutung für den Konzern Werbung Der Nachhaltigkeitsbericht Volkswagen 2022 markiert einen Wendepunkt in der Unternehmensstrategie des Konzerns....

Ähnlichster Bericht-Chunk 1 (Score: 0.85):
Sie sollen den Menschen eine Perspektive geben und in der Transformation eine klare Richtung für den Veränderungsprozess vorgeben. Die Beschäftigungssicherung bis 2029 in der Volkswagen AG gibt den Beschäftigten dabei den notwendigen Rückhalt. Verantwortung gesamtheitlich: die Lieferkette im Blick N
--------------------------------------------------------------------------------
Ähnlichster Bericht-Chunk 2 (Score: 0.84):
Der Volkswagen Konzern führt mit seinen Marken eine eng abgestimmte Technologie- und Produktplanung durch, um sowohl bestehende als auch sich verschärfende gesetzliche Vor gaben umzusetzen und Zielverfehlungen zu vermeiden. CO2-Bepr

## Klassifikation

In [14]:
from transformers import pipeline

# Modell laden
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Kategorien, auf die klassifiziert werden soll
labels = [
    "konkrete Maßnahme",
    "vage Behauptung",
    "fragwürdige Nachhaltigkeitsaussage"
]

# Klassifikation auf die gefilterten Bericht-Chunks anwenden
classification_results = []

for chunk in filtered_chunks:
    result = classifier(
        chunk,
        candidate_labels=labels,
        hypothesis_template="Dies ist eine {}."
    )
    
    classification_results.append({
        "text": chunk,
        "label": result["labels"][0],
        "score": result["scores"][0],
        "all_labels": result["labels"],
        "all_scores": result["scores"]
    })

Device set to use cpu


In [15]:
import pandas as pd

df_classified = pd.DataFrame(classification_results)
df_sorted = df_classified.sort_values(by="score", ascending=False)

# Nur die Top-10 Greenwashing-ähnlichsten anzeigen
df_greenwashing = df_sorted[df_sorted["label"] == "fragwürdige Nachhaltigkeitsaussage"]
print(df_greenwashing.head(10)[["text", "label", "score"]])

                                                  text  \
165  Potenziell sich daraus ergebende Nachfrage übe...   
437  Die Erkenntnisse der Analyse sollen zur Ableit...   
25   In den Kennzahlen und in den dazugehörigen Zie...   
446  Die Entwicklung des ge meinsamen Fragebogensta...   
100  GRI 2-28, 2-29 Inhalt Unternehmens-DNA | Dekar...   
137  Zu nennen sind hier beispielhaft Risiken, die ...   
310  Inhalt Unternehmens-DNA | Dekarbonisierung | K...   
33   Darüber hinaus kann es auch zu Abweichungen vo...   
371  Wirksamkeit messen Um die aktuelle Performance...   
166  Reputation Berichterstattung und Kommunikation...   

                                  label     score  
165  fragwürdige Nachhaltigkeitsaussage  0.806009  
437  fragwürdige Nachhaltigkeitsaussage  0.799571  
25   fragwürdige Nachhaltigkeitsaussage  0.753235  
446  fragwürdige Nachhaltigkeitsaussage  0.734970  
100  fragwürdige Nachhaltigkeitsaussage  0.731848  
137  fragwürdige Nachhaltigkeitsaussage  0.713258

In [16]:
df_classified["label"].value_counts()

label
fragwürdige Nachhaltigkeitsaussage    214
konkrete Maßnahme                     201
vage Behauptung                        50
Name: count, dtype: int64

In [17]:
df = pd.DataFrame(classification_results)
df["text_short"] = df["text"].str.slice(0, 150)
df.to_csv("greenwashing_classification.csv", index=False)

In [18]:
for label in df['label'].unique():
    print(f"\n{label} (Beispiele):")
    beispiele = df[df["label"] == label].sort_values(by="score", ascending=False).head(3)
    for i, row in beispiele.iterrows():
        print(f"   {i+1}. {row['text_short']}... (Score: {row['score']:.2f})")


fragwürdige Nachhaltigkeitsaussage (Beispiele):
   166. Potenziell sich daraus ergebende Nachfrage überhänge für Strom aus nichtfossilen Quellen können zu höheren Marktpreisen führen. Dadurch können Mehr ko... (Score: 0.81)
   438. Die Erkenntnisse der Analyse sollen zur Ableitung geeigneter Maßnahmen herangezogen werden. • Lieferantenmanagement: Die Bearbeitung des Themen bereic... (Score: 0.80)
   26. In den Kennzahlen und in den dazugehörigen Zielen bei den Themen Umweltentlastung Produktion (UEP), Dekarbo nisierungsindex (DKI), Stimmungsbarometer,... (Score: 0.75)

konkrete Maßnahme (Beispiele):
   179. Dekarbonisierungsprogramm nimmt gesamten Lebenszyklus in den Blick Unser umfassendes Dekarbonisierungsprogramm schließt den gesamten Lebenszyklus der ... (Score: 0.92)
   428. Seit 2022 Das S-Rating ist ein konzerneinheitliches Instrument, mit dem die Nachhaltigkeitsperformance relevanter Lieferanten2 in den Bereichen Umwelt... (Score: 0.91)
   408. Inhalt Unternehmens-DNA | Dekarb